In [57]:
import pandas as pd
import pylidc as pl
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import os
from radiomics import featureextractor
print(np.__version__)
import shutil

import sys
print(sys.executable)

#print(pl.__version__)

1.24.4
c:\Users\admin\Desktop\Laboratorios\.conda\python.exe


Codigo para lembrar apenas

In [ ]:
import sys
print(sys.executable)

Aqui elimina os pacientes que nao tem anotações (TÊM DE CORRER)

In [24]:
input_directory = "C:\\Users\\admin\\Desktop\\Laboratorios\\manifest-1600709154662\\LIDC-IDRI"

# List with all the eliminated patients - the one who were not evaluated
deleted_patients = []
# Ordered list of all the subfolders
patient_folders = sorted(os.listdir(input_directory))

for patient_folder in patient_folders:
    patient_folder_path = os.path.join(input_directory, patient_folder)

    # Id of the patient - 'LIDC-IDRI-xxxx'
    patient_id = os.path.basename(patient_folder_path)
    
    patient_scans = pl.query(pl.Scan).filter(pl.Scan.patient_id == patient_id)
    
    for scan in patient_scans:
        anns = scan.annotations
        
        if not anns:
            # Path to the folder we want to eliminate
            delete_folder = os.path.join(input_directory, patient_id) 

            # Verify if the folder exists
            if os.path.exists(delete_folder):
                # Verify if it is a folder
                if os.path.isdir(delete_folder):
                    # Remove recursively
                    shutil.rmtree(delete_folder)
                    print(f"The folder '{patient_id}' and its content were removed successfully")
                    deleted_patients.append(patient_id)
                else:
                    print(f"'{patient_id}' is not a folder.")
            else:
                print(f"The folder '{patient_id}' does not exist in the expecified path.")

# If desired - save the list on a file  
folder = "deleted_patients.txt"

with open(folder, 'w') as arquivo:
    # Saves the patients patient_id on a file named deleted_patients.txt
    for element in deleted_patients:
        arquivo.write(element + '\n')

# Leitura dos ficheiros

In [ ]:
nodule_counts = pd.read_excel('lidc-idri-nodule-counts-6-23-2015.xlsx', engine='openpyxl')
patient_diagnoses = pd.read_excel('tcia-diagnosis-data-2012-04-20.xls')
dicom_metadata = pd.read_csv('LIDC-IDRI_MetaData.csv')


# Inicializar pylidc para trabalhar com as anotações XML
cases = pl.query(pl.Scan).all()



print(nodule_counts.head())
print(100*"*")
print(patient_diagnoses.head())
print(100*"*")
print(dicom_metadata.head())
print(100*"*")
print(len(cases))

Renomear colunas

In [8]:
# Renomear as colunas para 'PatientID' se necessário
nodule_counts.rename(columns={'TCIA Patent ID': 'PatientID'}, inplace=True)
patient_diagnoses.rename(columns={'TCIA Patient ID': 'PatientID'}, inplace=True)
dicom_metadata.rename(columns={'Subject ID': 'PatientID'}, inplace=True)



# Merge dos datasets Nodule_counts , dicom_metadata , anotações (XML)

In [ ]:
patient_info = pd.merge( dicom_metadata,nodule_counts, on='PatientID', how='inner')
print(patient_info)



# Unificar com os metadados DICOM
complete_data = pd.merge(patient_info, patient_diagnoses, on='PatientID', how='inner')

print(complete_data.head())

# Criação do dataset com as anotações de cada nodulo

In [ ]:
'''
np.int = np.int32  # Garantir compatibilidade com versões mais novas do NumPy

# Criar uma lista para armazenar informações detalhadas dos contornos dos nódulos
nodule_data = []

# Iterar sobre todos os casos/pacientes
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0001').first()

patient_id = scan.patient_id
print(f"Processing Patient ID: {patient_id}")

# Iterar sobre os nódulos do paciente
# Iterar sobre os nódulos do paciente
for nodule_list in scan.cluster_annotations():
    if len(nodule_list) > 0:
        for nodule in nodule_list:
            print(f"Inspecting Nódulo {nodule.id}: {nodule}")
            
            # Armazenar o ID do contorno e a posição da fatia axial (Z_Position)
            nodule_data.append({
                'PatientID': patient_id,
                'NoduleID': nodule.id,
                'InternalStructure': nodule.internalStructure,
                'Sphericity': nodule.sphericity,
                'Calcification': nodule.calcification,
                'Subtlety': nodule.subtlety,
                'Lobulation': nodule.lobulation,
                'Margin': nodule.margin,
                'Spiculation': nodule.spiculation,
                'Texture': nodule.texture,
                'Malignancy': nodule.malignancy
            })
    else:
        print(f"Nódulo {nodule_list} está vazio.")

# Converter para DataFrame
nodule_df = pd.DataFrame(nodule_data)

nodule_df.to_csv('C:\\Users\\admin\\Desktop\\Laboratorios\\noduledf2.csv', index=False)

# Exibir o DataFrame
print(nodule_df)
'''

# Transformas os dois datasets em csv

In [12]:
#nodule_df.to_csv('C:\\Users\\admin\\Desktop\\Laboratorios\\nodule_df.csv', index=False)
#complete_data.to_csv('C:\\Users\\gjoli\\Desktop\\UNI3\\lab\\complete_data.csv', index=False)

AHSDSAHDUASHDUAHSD NORMALIZAÇÂO (Nao esta a funcionar )

In [26]:
def normalize(volume, intercept, slope):
    """ Normaliza o volume de acordo com os valores de intercept e slope fornecidos pelo DICOM """
    volume = volume.astype(np.float32)
    volume = volume * slope + intercept
    # Limitar os valores em uma faixa significativa, como -1000 a 400, que é típico para pulmões
    volume = np.clip(volume, -1000, 400)
    return volume


# Aqui começa a cena de sacar as coordenadas e isolar os nodulos

In [41]:
def get_nodule_coordinates(scan, patient_id): 
    # Garantir que np.int e np.bool estejam definidos corretamente
    np.int = np.int32
    np.bool = np.bool_

    # Verificar se o scan está vazio
    if scan is None:
        print(f"Nenhum scan encontrado para o paciente {patient_id}")
        return None, None, None
    
    # Obter as anotações dos nódulos
    nodules = scan.cluster_annotations()
    
    # Verificar se há nódulos no scan
    if len(nodules) == 0:
        print(f"Nenhum nódulo encontrado para o paciente {patient_id}")
        return None, None, None
    '''
    # Extrair intercept e slope da primeira fatia DICOM
    try:
        dicom_slice = scan.load_all_dicom_images()[0]  # Carrega a primeira fatia
        intercept = dicom_slice.RescaleIntercept
        slope = dicom_slice.RescaleSlope
    except AttributeError:
        print("Não foi possível obter intercept e slope das fatias DICOM")
        return None, None, None
    '''
    vec_masks = []
    vol = scan.to_volume()  # Inicializar o volume fora do loop

    # Verificar se o volume foi corretamente carregado
    if vol is None:
        print(f"Erro ao carregar o volume para o paciente {patient_id}")
        return None, None, None

    # Normalizar o volume
    # vol = normalize(vol, intercept, slope)

    # Percorrer cada nódulo anotado
    for annotation in nodules:
        # Inicializar uma máscara vazia com o mesmo shape do volume
        consensus_mask_full = np.zeros_like(vol, dtype=bool)

        # Percorrer os contornos do nódulo
        for contour in annotation:
            # Obter as coordenadas da caixa delimitadora (slices) para o contorno
            slices = contour.bbox()

            # Criar a máscara do contorno
            mask = contour.boolean_mask()

            if consensus_mask_full[slices].shape == mask.shape:
                consensus_mask_full[slices] |= mask  # Combine as máscaras se os shapes forem iguais
            else:
                print(f"Shapes incompatíveis: {consensus_mask_full[slices].shape} e {mask.shape}")

            # Inserir a máscara da anotação na posição correta dentro da máscara completa
            #consensus_mask_full[slices] |= mask  # Combinar as máscaras usando OR lógico

        vec_masks.append(consensus_mask_full)

    # Retornar as máscaras, o volume e o ID do último contorno processado
    return vec_masks, vol, contour.id


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import draw

import numpy as np

def isolate_nodule_with_coordinates(consensus_mask_full,vol):
    
    # Encontrar a fatia com a maior área do nódulo
    slice_areas = consensus_mask_full.sum(axis=(0, 1))
    max_slice_idx = np.argmax(slice_areas)

    # Obter a imagem e a máscara da fatia correspondente
    img_slice = vol[:, :, max_slice_idx]
    mask_slice = consensus_mask_full[:, :, max_slice_idx]

    return img_slice,mask_slice, max_slice_idx


'''
# Exemplo de uso da função
# Vamos supor que 'pixel_array' é a imagem e 'coordinates' são as coordenadas fornecidas

# Aqui 'pixel_array' seria a tua imagem DICOM carregada
# coordinates seria o array de coordenadas que mencionaste
isolated_nodule = isolate_nodule_with_coordinates(pixel_array, coordinates)

# Visualizar o nódulo isolado
plt.imshow(isolated_nodule, cmap='gray')
plt.title("Nódulo Isolado")
plt.show()
'''

In [15]:
from skimage.transform import resize

def normalize_image(image, target_shape=(512, 512)):
    """
    Normaliza uma imagem para uma forma padrão, ajustando a escala e a dimensão.

    Parameters:
    - image: ndarray
        A imagem original a ser normalizada.
    - target_shape: tuple
        A forma desejada da imagem normalizada (default é (512, 512)).

    Returns:
    - normalized_image: ndarray
        A imagem normalizada.
    """
    # Verifica se a imagem precisa de redimensionamento
    if image.shape[:2] != target_shape:
        # Usar a função resize da skimage para redimensionar corretamente
        normalized_image = resize(image, target_shape, preserve_range=True)
    else:
        normalized_image = image
    
    return normalized_image

## Aqui em baixo são as duas funções de guardar as máscaras. A segunda guarda a máscara em binário a primeira guarda em DICOM acho eu

In [17]:
def save_mask(image, patient_id, nodule_id, sop_instance_uid, base_masks_dir):
    # Criar o diretório com base no PatientID e NoduleID
    save_dir = os.path.join(base_masks_dir, str(patient_id), str(nodule_id))
    os.makedirs(save_dir, exist_ok=True)  # Cria as pastas se não existirem
    
    # Salvar a imagem no formato .npy
    file_path = os.path.join(save_dir, f'{sop_instance_uid}.npy')  # Nome do arquivo com SOPInstanceUID
    np.save(file_path, image)
    
    print(f'Salvo: {file_path}')


Codigo para fazer as mascaras. Se quiserem imprimir as imagens tirem o hastag dos plts e assim. Se quiserem guardar as mascaras tirem o hastag do save mask

In [ ]:
def isolate_nodules(base_masks_dir):
    # Verificar se o ID do paciente é menor ou igual a 57
    for i in range(1, 55):  # De 1 até 55 (incluindo)
        patient_id = f"LIDC-IDRI-{i:04d}"  # Formato do ID com zeros à esquerda

        print(patient_id)

        if patient_id =="LIDC-IDRI-0028" or patient_id =="LIDC-IDRI-0032":
            continue

        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == patient_id).first()

        #patient_dir = os.path.join(base_dir, patient_id)

        dicom_files = scan.load_all_dicom_images()

        #print(patient_dir)

        # Obter as coordenadas do nódulo
        vec_masks, vol, nodule_id = get_nodule_coordinates(scan,patient_id)
        
        if vec_masks is None or vol is None:
            print(f"Nódulo ou volume não encontrado para {patient_id}.")
            continue
        
        #print("Comprimento: ", len(vec_masks))

        for i in range (len(vec_masks)):

            #print("Iteracao: ", i)

            # Isolar o nódulo usando as coordenadas
            img_slice, mask_slice, max_slice = isolate_nodule_with_coordinates(vec_masks[i], vol)

            #print(max_slice)
            #print(len(dicom_files))

            #print(max_slice)

            dicom_file_name = dicom_files[max_slice].SOPInstanceUID

            #print(dicom_file_name)

            # Criar uma figura com dois subplots
            #fig, axes = plt.subplots(1, 2, figsize=(16, 8))

            # Exibir a imagem original
            #axes[0].imshow(img_slice, cmap='gray')
            #axes[0].contour(mask_slice, colors='r')
            #axes[0].set_title('Original')
            #axes[0].axis('off')

            # Exibir a imagem com o contorno
            #axes[1].imshow(mask_slice, cmap='gray')
            #axes[1].set_title('Contorno')
            #axes[1].axis('off')

            # Mostrar a figura
            #plt.show()

            #save_mask(mask_slice, patient_id, nodule_id,dicom_file_name, base_masks_dir)

# Usar a função
base_dir = "C:\\Users\\admin\\Desktop\\Laboratorios\\manifest-1600709154662\\LIDC-IDRI"
base_masks_dir = "C:\\Users\\admin\\Desktop\\Laboratorios\\Masks"
isolate_nodules(base_masks_dir)


# A partir daqui começa o radiomics



In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
import pydicom
from radiomics import featureextractor
import pylidc as pl

# Função para buscar o índice do arquivo DICOM correspondente pelo SOPInstanceUID
def find_dicom_file(dicom_files, sop_instance_uid):
    for i in range(len(dicom_files)):
        dicom_file_name = dicom_files[i].SOPInstanceUID
        if sop_instance_uid == dicom_file_name:
            print("DICOM FILE NAME: ", dicom_file_name)
            return i  # Retorna o índice do arquivo
    return None

# Caminho base para as máscaras
base_mask_path = "C:\\Users\\admin\\Desktop\\Laboratorios\\Masks"

# Caminho do arquivo de parâmetros YAML
params_file = "C:\\Users\\admin\\Desktop\\Laboratorios\\Params.yaml"

# Inicializar o extrator de features
extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# Loop sobre as máscaras
for root, dirs, files in os.walk(base_mask_path):
    for file in files:
        if file.endswith(".npy"):
            mask_path = os.path.join(root, file)

            # Extração das informações da máscara a partir do caminho
            path_parts = mask_path.split(os.sep)
            patient_id = path_parts[-3]  # PatientID
            nodule_id = path_parts[-2]   # NoduleID
            sop_instance_uid = file.replace(".npy", "")  # Usar o nome do arquivo npy como SOPInstanceUID

            # Buscar o scan do paciente
            scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == patient_id).first()

            # Carregar todas as imagens DICOM do paciente
            dicom_files = scan.load_all_dicom_images()

            # Buscar o índice do arquivo DICOM correspondente usando o SOPInstanceUID
            dicom_file_index = find_dicom_file(dicom_files, sop_instance_uid)

            if dicom_file_index is not None:
                # Obter o arquivo DICOM correspondente pelo índice
                dicom_file = dicom_files[dicom_file_index]

                # Ler a imagem DICOM
                pixel_array = dicom_file.pixel_array

                # Normalizar a imagem
                pixel_array = normalize_image(pixel_array)

                intercept = dicom_file.RescaleIntercept
                slope = dicom_file.RescaleSlope

                # Converter para Hounsfield Units
                img_hu = pixel_array * slope + intercept

                # Carregar a máscara npy
                mask = np.load(mask_path)

                # Converter a imagem e a máscara para SimpleITK
                img_sitk = sitk.GetImageFromArray(img_hu.astype(np.float32))
                mask_sitk = sitk.GetImageFromArray(mask.astype(np.uint8))

                # Extrair as features usando o PyRadiomics
                features = extractor.execute(img_sitk, mask_sitk, label=1)
                dic_list.append(features)

                # Adicionar PatientID e NoduleID às features
                features['PatientID'] = patient_id
                features['NoduleID'] = nodule_id

            else:
                print(f"Arquivo DICOM correspondente para {mask_path} não encontrado.")

# Criar DataFrame a partir das features coletadas
features_df = pd.DataFrame(dic_list)

# Salvar as features extraídas em um arquivo CSV
output_csv = f"C:/Users/admin/Desktop/Laboratorios/Featurestodas/features_radiomics.csv"
features_df.to_csv(output_csv, index=False)
